In [11]:
PATH = r''
file_name = 'suspicious_tweets_off.csv'
import os
os.chdir(PATH)

In [ ]:
%run setup.ipynb

In [13]:
dataset = []

with open(file_name, 'r') as file:
    reader = csv.reader(file)
    header = next(reader)
    for row in reader:
        row[1] = int(row[1])
        dataset.append(row)

print(dataset[:5])

[['awww bummer shoulda get david carr third day', 1], ['upset update facebook texting might cry result school today also blah', 1], ['dive many time ball manage save rest bound', 1], ['whole body feel itchy like fire', 0], ['hey long time see yes rain bit bit lol fine thank', 1]]


Creation of a training set and a test set:

In [14]:
docs = [row[0] for row in dataset]
labels = [[row[1]] for row in dataset]
train_set, test_set, train_labels, test_labels = train_test_split(docs, labels, test_size=0.2, random_state=42)

Creation of the corpus:

In [15]:
corpus = Corpus()

for doc_text, label in zip(train_set, train_labels):
    words = word_tokenize(doc_text)
    corpus.add_doc(words, y=label)

Definition of a function that compute the optimal value of the threshold, based on the F1-score, used for the binary convertion in the next function.

In [16]:
def find_opt_tresh(thresholds, prob, test_labels):
    opt_t = None
    binary_predictions_def = None
    max_f1 = 0
    for t in thresholds:
        binary_predictions = (prob > t).astype(int)
        f1_temp = f1_score(test_labels, binary_predictions, average='weighted')
        if f1_temp > max_f1:
            opt_t = t
            binary_predictions_def = binary_predictions
            max_f1 = f1_temp
    return opt_t, binary_predictions_def

Defining a function that return the sLDA model with the optimal number of topics, based on the F1-score, and the binary prediction on the test set.

In [20]:
def find_opt_slda(n_topics, n_thresh, test_set, test_labels):
    max_f1 = 0
    opt_k = None
    opt_t = None
    binary_predictions_def = None
    slda_model_def = None

    for k in n_topics:
        slda_model = tp.SLDAModel(vars=['b'], k=k)
        slda_model.add_corpus(corpus) #train in two steps
        slda_model.train(1000, show_progress=True)

        # regression label = topic distribution for a given document
        coef = slda_model.get_regression_coef(0) # extracts regression coef
        test_docs = [slda_model.make_doc(word_tokenize(doc)) for doc in test_set] # list of test documents 
        test_predictions = [slda_model.infer(doc) for doc in test_docs]  # document-topic prob distributions
        y_test = [np.dot(coef, test_predictions[i][0]) for i in range(len(test_predictions))]
        prob = scipy.special.expit(y_test) # applies sigmoid function 

        current_opt_t, binary_predictions = find_opt_tresh(n_thresh, prob, test_labels)
        temp_f1 = f1_score(test_labels, binary_predictions, average='weighted')

        if temp_f1 > max_f1:
            max_f1 = temp_f1
            opt_k = k
            opt_t = current_opt_t
            binary_predictions_def = binary_predictions
            slda_model_def = slda_model

    print(f'Optimal number of topics: {opt_k}')
    print(f'Optimal threshold: {opt_t}')
    
    return slda_model_def, binary_predictions_def

In [21]:
n_topics = range(10,101,10)
n_thresh = np.round(np.arange(0.5, 1, 0.05), 2)
slda_model, binary_predictions = find_opt_slda(n_topics, n_thresh, test_set, test_labels)

Iteration: 100%|██████████| 1000/1000 [01:19<00:00, 12.53it/s, LLPW: -9.005730]


Optimal number of topics: 20
Optimal threshold: 0.8


Computing different metrics:

In [22]:
# Accuracy
accuracy = accuracy_score(test_labels, binary_predictions)
print(f'Accuracy: {accuracy}')

# F1-score
f1 = f1_score(test_labels, binary_predictions, average='weighted')
print(f'F1-score: {f1}')

# Function to compute recall and precision
def calculate_recall_precision(true_labels, predicted_labels, positive_class=1):
    true_positives = sum((true[0] == positive_class) and (pred == positive_class) for true, pred in zip(true_labels, predicted_labels))
    false_positives = sum((true[0] != positive_class) and (pred == positive_class) for true, pred in zip(true_labels, predicted_labels))
    false_negatives = sum((true[0] == positive_class) and (pred != positive_class) for true, pred in zip(true_labels, predicted_labels))

    recall = true_positives / (true_positives + false_negatives) if (true_positives + false_negatives) > 0 else 0.0
    precision = true_positives / (true_positives + false_positives) if (true_positives + false_positives) > 0 else 0.0

    return recall, precision

# Recall and Precision
recall, precision = calculate_recall_precision(test_labels, binary_predictions)
print(f'Recall: {recall:.2f}')
print(f'Precision: {precision:.2f}')

Accuracy: 0.949775194648536
F1-score: 0.9503088732460222
Recall: 0.97
Precision: 0.97


In [ ]:
tmp_plot = tmp.report(model=slda_model, docs=train_set)
tmp_plot